Per trovare i parametri degli altri settori basta cambiare il nome del settore (quindi CTRL+H, sostituire per esempio Calcio con Pesca e rieseguire il codice)

In [ ]:
datasetAR = pd.read_csv("serie-storiche-ecommerce-ARIMA.csv", sep =",")
pd.set_option('display.max_rows', None)

In [ ]:
!pip install pmdarima
from pmdarima import auto_arima # for determining ARIMA orders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 8.7 MB/s 
     |████████████████████████████████| 9.8 MB 49.2 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
datasetAR.index.name = "data"
#print(datasetAR.index.name)
datasetAR["totale"] = datasetAR["data"] + "." + datasetAR["totale"] #unione delle colonne in quanto l'import del dataset riscontra dei problemi 
datasetAR["totale"]= datasetAR["totale"].str.replace("'","")
datasetAR=datasetAR.drop(columns="data")
#display(datasetAR)

In [ ]:
x = datasetAR["settore"].unique() #inserimento nell'array dei possibili settori
x=x.tolist() #trasformazione da array a lista
index=15 #in posizione 15 c'è il valore Nan che deve essere eliminato
x.pop(index) #viene eliminato il valore Nan 
#print(x)

#ciclo per sistemare i valori che contengono Nan andando ad inserire il Settore e il Totale corretto 
for index, row in datasetAR.iterrows():
    if (row['settore'] not in x):
            row['settore'] = row['totale'].split('.')[1]
            row['totale'] = row['totale'].split('.')[0]
datasetAR["totale"]=datasetAR["totale"].astype(float)
datasetAR = datasetAR.reset_index()
#display(datasetAR) 

In [ ]:
dataCalcioAR = datasetAR[datasetAR['settore'] == "Calcio"]

In [ ]:
!pip install cython
!pip install pmdarima
!pip install cuda-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.4 MB 6.8 MB/s 


In [ ]:
dataCalcioAR = dataCalcioAR.set_index('data')
dataCalcioAR.index = pd.to_datetime(dataCalcioAR.index)

In [ ]:
from statsmodels.tools.eval_measures import rmse
from pmdarima.preprocessing import FourierFeaturizer

import warnings
warnings.filterwarnings("ignore")

def rollingDaysFourier(data, train_end, window, iter = 10):

  rmseList = list()
  
  train_data = data[:train_end]
  
# definizione del parametro i di Fourier
  aic_best = np.inf
  len_k = 0

  for i in range(20):
    faulty = False
    print("try k = "+str(i))
    train_data = data[:train_end]
    trans = FourierFeaturizer(365, i) # i=1
    try:
      y_prime, X = trans.fit_transform(train_data["totale"].to_numpy())
    except:
      faulty = True
    if faulty:
      pass
    else:
      stepwise_fit = auto_arima(train_data, seasonal=False, exogenous=X, error_action='ignore', suppress_warnings=True, trace =True)
      aic_value = stepwise_fit.aic()
      if aic_value < aic_best:
        aic_best =  aic_value
        best_fit = stepwise_fit
        len_k = i 
      else:
        pass

  stepwise_fit = best_fit

  for i in range(iter):
    test_end = train_end+timedelta(days=window)
    test_data = data[train_end + timedelta(days=1):test_end]
    X_test = X[train_end + timedelta(days=1):test_end]
    train_end += timedelta(days=1)

    print(stepwise_fit.order)
    print(stepwise_fit.seasonal_order)
    # #model = SARIMAX(train_data, order=stepwise_fit.order, exog=X_train)
    # #fit the model
    # start = time()
    # #model_fit = stepwise_fit.fit()
    # end = time()
    # print('Model Fitting Time:', end - start)
    # #get the predictions and residuals
    predictions = stepwise_fit.predict(window, X=X_test)
    predictions = pd.DataFrame({'date':test_data.index[0] ,'totale':predictions})
    residuals = test_data.totale - predictions.totale
    predictions = predictions.set_index('date')
    # predictions.index = pd.to_datetime(predictions.index)
    # print('Mean Absolute Percent Error:', round(np.mean(abs(residuals/test_data.totale)),4))
    # print('Root Mean Squared Error:', (rmse(test_data['totale'], predictions['totale'])))
    print('real value', test_data)
    print('predictions', predictions)
    
    EPSILON = 1e-10
    rmspe = (np.sqrt(np.mean(np.square((test_data['totale'] - predictions['totale']) / (test_data['totale'] + EPSILON)), axis=0))) * 100
    print(rmspe)
    rmseList.append(rmspe)

  return rmseList

In [ ]:
dataCalcioAR = dataCalcioAR.drop(columns=['settore'])

In [ ]:
train_end = datetime(2021,12,31)
rmseARIMAFourier = rollingDaysFourier(dataCalcioAR,train_end, 1, 14)
print(rmseARIMAFourier)